In [ ]:
import pandas as pd
import os, json
import xgboost as xgb
import lightgbm as lgb
import numpy as np
from optuna.samplers import TPESampler

In [ ]:
import optuna

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm
import sklearn.neural_network
import sklearn.tree
import sklearn.metrics

In [ ]:
with open("connectives_dict.json") as f:
    connective_dict = json.load(f)
connective_dict["20"]

In [ ]:
# def filter_columns(df, connective_selection):
#     meta_cols = list()
#     cols_conn = list()
#     for col in df.columns:
#         if col == "index":
#             pass
#         elif not "bert" in col:
#             meta_cols.append(col)
#         else:
#             model_name, connective = col.split("_")
#             if connective in connective_selection:
#                 cols_conn.append(col)
#     return df[meta_cols+cols_conn]

In [ ]:
for filename in [f for f in os.listdir("features/all_conns_roberta/all_conns/") if f.endswith(".csv")]:
    df = pd.read_csv("features/all_conns_roberta/all_conns/"+filename)
    for perc in ["all", "80", "40", "20"]:
        for name, split_perc, split_conn in [(perc+"_attack_support/", perc, "attack+support"),  (perc+"_attack/", perc, "attack"),  (perc+"_support/", perc, "support"),  (perc+"_both/", perc, "both")]:

            if split_conn == "attack+support":
                df_new = filter_columns(df, connective_dict[split_perc]["attack"]+connective_dict[split_perc]["support"])
            else:
                if split_conn in connective_dict[split_perc]:
                    df_new = filter_columns(df, connective_dict[split_perc][split_conn])
                    # print(connective_dict[split_perc][split_conn])
            print(filename, name, split_perc, split_conn, df_new.shape)
            if not os.path.exists("features/all_conns_roberta/"+name):
                os.makedirs("features/all_conns_roberta/"+name)
            df_new.to_csv("features/all_conns_roberta/"+name+filename,index=False)


In [ ]:
# for filename in [f for f in os.listdir("features/all_conns_bert/all_conns/") if f.endswith(".csv")]:
#     df = pd.read_csv("features/all_conns_bert/all_conns/"+filename)
#     for perc in ["all", "80", "40", "20"]:
#         for name, split_perc, split_conn in [(perc+"_attack_support/", perc, "attack+support"),  (perc+"_attack/", perc, "attack"),  (perc+"_support/", perc, "support"),  (perc+"_both/", perc, "both")]:

#             if split_conn == "attack+support":
#                 df_new = filter_columns(df, connective_dict[split_perc]["attack"]+connective_dict[split_perc]["support"])
#             else:
#                 if split_conn in connective_dict[split_perc]:
#                     df_new = filter_columns(df, connective_dict[split_perc][split_conn])
#                     # print(connective_dict[split_perc][split_conn])
#             print(filename, name, split_perc, split_conn, df_new.shape)
#             # if not os.path.exists("features/all_conns_bert/"+name):
#             #     os.makedirs("features/all_conns_bert/"+name)
#             # df_new.to_csv("features/all_conns_bert/"+name+filename,index=False)


In [ ]:
def detailed_objective_test(trial):
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "MLPClassifier", "DecisionTreeClassifier", "RandomForestClassifier", "XGBoost", "LightGBM"])
    # classifier_name = "LightGBM"
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-3, 1e3, log=True)
        svc_kernel = trial.suggest_categorical("svc_kernel", ["rbf", "linear", "poly"])
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto", kernel=svc_kernel)
    elif classifier_name == "RandomForestClassifier":
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        rf_estimators = trial.suggest_int("rf_estimator", 10, 100)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_estimators
        )
    elif classifier_name == "MLPClassifier":
        mlp_activation = trial.suggest_categorical("mlp_activation", ["tanh", "relu", "logistic", "identity"])
        mlp_solver = trial.suggest_categorical("mlp_solver", ["adam", "lbfgs", "sgd"])
        # first_layer_neurons = trial.suggest_int('first_layer_neurons', 10, 100, step=10)
        # second_layer_neurons = trial.suggest_int('second_layer_neurons', 10, 100, step=10)
        n_layers = trial.suggest_int('n_layers', 1, 4)
        layers = []
        for i in range(n_layers):
            layers.append(trial.suggest_int('n_units_l_{}'.format(i), 10, 100, step=10))
        classifier_obj = sklearn.neural_network.MLPClassifier(activation=mlp_activation, solver=mlp_solver, max_iter=500, hidden_layer_sizes=tuple(layers))
    elif classifier_name == "DecisionTreeClassifier":
        dct_criterion = trial.suggest_categorical("dct_criterion", ["gini", "entropy", "log_loss"])
        dct_max_depth = trial.suggest_int("dct_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.tree.DecisionTreeClassifier(criterion=dct_criterion, max_depth=dct_max_depth)
    elif classifier_name == "XGBoost":
        dtrain = xgb.DMatrix(df_train[feat_cols], label=df_train["stance"])
        dvalid = xgb.DMatrix(df_test[feat_cols], label=df_test["stance"])

        param = {
            "verbosity": 0,
            "objective": "binary:logistic",
            # use exact for small dataset.
            "tree_method": "exact",
            # defines booster, gblinear for linear functions.
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        }

        if param["booster"] in ["gbtree", "dart"]:
            # maximum depth of the tree, signifies complexity of the tree.
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            # minimum child weight, larger the term more conservative the tree.
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            # defines how selective algorithm is.
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    elif classifier_name == "LightGBM":
        dtrain = lgb.Dataset(df_train[feat_cols], label=df_train["stance"])

        param = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0, step=0.1),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0, step=0.1),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 3),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=10),
        }

    if classifier_name == "XGBoost":
        bst = xgb.train(param, dtrain)
        preds_prec = bst.predict(dvalid)
        preds = np.rint(preds_prec)
    elif classifier_name == "LightGBM":
        gbm = lgb.train(param, dtrain)
        preds_prec = gbm.predict(df_test[feat_cols])
        preds = np.rint(preds_prec)
    else:
        classifier_obj.fit(df_train[feat_cols], df_train["stance"])
        preds = classifier_obj.predict(df_test[feat_cols])

    acc = sklearn.metrics.accuracy_score(preds, df_test["stance"])
    precision, recall, f1, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_test["stance"], y_pred=preds, average="macro")
    precision_per_label, recall_per_label, f1_per_label, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_test["stance"], y_pred=preds, average=None, labels=[0,1])

    return acc, f1, recall, precision, f1_per_label, recall_per_label, precision_per_label

In [ ]:
def objective_optuna(trial):

    classifier_name = trial.suggest_categorical("classifier", ["SVC", "MLPClassifier", "DecisionTreeClassifier", "RandomForestClassifier", "XGBoost", "LightGBM"])
    # classifier_name = "LightGBM"
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-3, 1e3, log=True)
        svc_kernel = trial.suggest_categorical("svc_kernel", ["rbf", "linear", "poly"])
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto", kernel=svc_kernel)
    elif classifier_name == "RandomForestClassifier":
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        rf_estimators = trial.suggest_int("rf_estimator", 10, 100)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_estimators
        )
    elif classifier_name == "MLPClassifier":
        mlp_activation = trial.suggest_categorical("mlp_activation", ["tanh", "relu", "logistic", "identity"])
        mlp_solver = trial.suggest_categorical("mlp_solver", ["adam", "lbfgs", "sgd"])
        # first_layer_neurons = trial.suggest_int('first_layer_neurons', 10, 100, step=10)
        # second_layer_neurons = trial.suggest_int('second_layer_neurons', 10, 100, step=10)
        n_layers = trial.suggest_int('n_layers', 1, 4)
        layers = []
        for i in range(n_layers):
            layers.append(trial.suggest_int('n_units_l_{}'.format(i), 10, 100, step=10))
        classifier_obj = sklearn.neural_network.MLPClassifier(activation=mlp_activation, solver=mlp_solver, max_iter=500, hidden_layer_sizes=tuple(layers))
    elif classifier_name == "DecisionTreeClassifier":
        dct_criterion = trial.suggest_categorical("dct_criterion", ["gini", "entropy", "log_loss"])
        dct_max_depth = trial.suggest_int("dct_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.tree.DecisionTreeClassifier(criterion=dct_criterion, max_depth=dct_max_depth)
    elif classifier_name == "XGBoost":
        dtrain = xgb.DMatrix(df_train[feat_cols], label=df_train["stance"])
        dvalid = xgb.DMatrix(df_dev[feat_cols], label=df_dev["stance"])

        param = {
            "verbosity": 0,
            "objective": "binary:logistic",
            # use exact for small dataset.
            "tree_method": "exact",
            # defines booster, gblinear for linear functions.
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        }

        if param["booster"] in ["gbtree", "dart"]:
            # maximum depth of the tree, signifies complexity of the tree.
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            # minimum child weight, larger the term more conservative the tree.
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            # defines how selective algorithm is.
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    elif classifier_name == "LightGBM":
        dtrain = lgb.Dataset(df_train[feat_cols], label=df_train["stance"])

        param = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0, step=0.1),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0, step=0.1),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 3),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=10),
        }



    # score = sklearn.model_selection.cross_val_score(classifier_obj, df_train[feat_cols], df_train["stance"], n_jobs=-1, cv=3)
    # accuracy = score.mean()
    # # todo use something else then mean?
    # return accuracy
    if classifier_name == "XGBoost":
        bst = xgb.train(param, dtrain)
        preds_prec = bst.predict(dvalid)
        preds = np.rint(preds_prec)
    elif classifier_name == "LightGBM":
        gbm = lgb.train(param, dtrain)
        preds_prec = gbm.predict(df_dev[feat_cols])
        preds = np.rint(preds_prec)
    else:
        classifier_obj.fit(df_train[feat_cols], df_train["stance"])
        preds = classifier_obj.predict(df_dev[feat_cols])
    # probs = clf.predict_proba(X_valid)
    
    f1_macro = sklearn.metrics.f1_score(y_true=df_dev["stance"], y_pred=preds, average="macro")
    return f1_macro


In [ ]:
def detailed_objective_dev(trial):
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "MLPClassifier", "DecisionTreeClassifier", "RandomForestClassifier", "XGBoost", "LightGBM"])
    # classifier_name = "LightGBM"
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-3, 1e3, log=True)
        svc_kernel = trial.suggest_categorical("svc_kernel", ["rbf", "linear", "poly"])
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto", kernel=svc_kernel)
    elif classifier_name == "RandomForestClassifier":
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        rf_estimators = trial.suggest_int("rf_estimator", 10, 100)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_estimators
        )
    elif classifier_name == "MLPClassifier":
        mlp_activation = trial.suggest_categorical("mlp_activation", ["tanh", "relu", "logistic", "identity"])
        mlp_solver = trial.suggest_categorical("mlp_solver", ["adam", "lbfgs", "sgd"])
        # first_layer_neurons = trial.suggest_int('first_layer_neurons', 10, 100, step=10)
        # second_layer_neurons = trial.suggest_int('second_layer_neurons', 10, 100, step=10)
        n_layers = trial.suggest_int('n_layers', 1, 4)
        layers = []
        for i in range(n_layers):
            layers.append(trial.suggest_int('n_units_l_{}'.format(i), 10, 100, step=10))
        classifier_obj = sklearn.neural_network.MLPClassifier(activation=mlp_activation, solver=mlp_solver, max_iter=500, hidden_layer_sizes=tuple(layers))
    elif classifier_name == "DecisionTreeClassifier":
        dct_criterion = trial.suggest_categorical("dct_criterion", ["gini", "entropy", "log_loss"])
        dct_max_depth = trial.suggest_int("dct_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.tree.DecisionTreeClassifier(criterion=dct_criterion, max_depth=dct_max_depth)
    elif classifier_name == "XGBoost":
        dtrain = xgb.DMatrix(df_train[feat_cols], label=df_train["stance"])
        dvalid = xgb.DMatrix(df_dev[feat_cols], label=df_dev["stance"])

        param = {
            "verbosity": 0,
            "objective": "binary:logistic",
            # use exact for small dataset.
            "tree_method": "exact",
            # defines booster, gblinear for linear functions.
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        }

        if param["booster"] in ["gbtree", "dart"]:
            # maximum depth of the tree, signifies complexity of the tree.
            param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
            # minimum child weight, larger the term more conservative the tree.
            param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            # defines how selective algorithm is.
            param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    elif classifier_name == "LightGBM":
        dtrain = lgb.Dataset(df_train[feat_cols], label=df_train["stance"])

        param = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0, step=0.1),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0, step=0.1),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 3),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 100, step=10),
        }



    # score = sklearn.model_selection.cross_val_score(classifier_obj, df_train[feat_cols], df_train["stance"], n_jobs=-1, cv=3)
    # accuracy = score.mean()
    # # todo use something else then mean?
    # return accuracy
    if classifier_name == "XGBoost":
        bst = xgb.train(param, dtrain)
        preds_prec = bst.predict(dvalid)
        preds = np.rint(preds_prec)
    elif classifier_name == "LightGBM":
        gbm = lgb.train(param, dtrain)
        preds_prec = gbm.predict(df_dev[feat_cols])
        preds = np.rint(preds_prec)
    else:
        classifier_obj.fit(df_train[feat_cols], df_train["stance"])
        preds = classifier_obj.predict(df_dev[feat_cols])

    acc = sklearn.metrics.accuracy_score(preds, df_dev["stance"])
    precision, recall, f1, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_dev["stance"], y_pred=preds, average="macro")
    precision_per_label, recall_per_label, f1_per_label, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_dev["stance"], y_pred=preds, average=None, labels=[0,1])

    return acc, f1, recall, precision, f1_per_label, recall_per_label, precision_per_label

In [ ]:
def train_eval_lightgbm(params, setting):
    dtrain = lgb.Dataset(df_train[feat_cols], label=df_train["stance"])
    param = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": params["lambda_l1"],
            "lambda_l2": params["lambda_l2"],
            "num_leaves": params["num_leaves"],
            "feature_fraction": params["feature_fraction"],
            "bagging_fraction": params["bagging_fraction"],
            "bagging_freq": params["bagging_freq"],
            "min_child_samples": params["min_child_samples"],
        }
    gbm = lgb.train(param, dtrain)
    if setting == "dev":
        preds_prec = gbm.predict(df_dev[feat_cols])
        preds = np.rint(preds_prec)
        acc = sklearn.metrics.accuracy_score(preds, df_dev["stance"])
        precision, recall, f1, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_dev["stance"], y_pred=preds, average="macro")
        precision_per_label, recall_per_label, f1_per_label, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_dev["stance"], y_pred=preds, average=None, labels=[0,1])
    elif setting == "test":
        preds_prec = gbm.predict(df_test[feat_cols])
        preds = np.rint(preds_prec)
        acc = sklearn.metrics.accuracy_score(preds, df_test["stance"])
        precision, recall, f1, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_test["stance"], y_pred=preds, average="macro")
        precision_per_label, recall_per_label, f1_per_label, support = sklearn.metrics.precision_recall_fscore_support(y_true=df_test["stance"], y_pred=preds, average=None, labels=[0,1])
        
    
    
    return acc, f1, recall, precision, f1_per_label, recall_per_label, precision_per_label

In [ ]:
best_parametr_mean = {"classifier": "LightGBM", "lambda_l1": 0.025, "lambda_l2": 0.05, "num_leaves": 200, "feature_fraction": 0.8, "bagging_fraction": 0.8, "bagging_freq": 2, "min_child_samples": 48}
best_parameter_median = {"classifier": "LightGBM", "lambda_l1": 0.0001, "lambda_l2": 0.002, "num_leaves": 220, "feature_fraction": 0.9, "bagging_fraction": 0.8, "bagging_freq": 2, "min_child_samples": 50}

In [ ]:
best_results = list()
best_params = best_parameter_median
search_params = False
feat_names = ["bert-base", "bert-large", "distilbert", "all_lms"]
for split_perc in ["all", "80", "40"]:  # "40", 
    for model_conns in ["all_conns_roberta"]:  # , "all_conns_bert"]:
        if model_conns == "all_conns_roberta":
            feat_names.extend(["roberta-base", "roberta-large", "xlm-roberta-base", "xlm-roberta-large"])
        for split_conn in ["attack_support", "attack", "support"]:  # , "all"]:
            for filename in ["ibmcs_all.csv", "argmin_all.csv", "perspectrum_all.csv"]:  # , "ibmargq_all.csv"]:  
                for feat_name in feat_names:
                    # for i in range(0,5):
                    df = pd.read_csv("features/"+model_conns+"/"+split_perc+"_"+split_conn+"/"+filename)
                    if feat_name == "all_lms":
                        feat_cols = [col for col in df.columns if "bert" in col]
                    else: 
                        feat_cols = [col for col in df.columns if col.startswith(feat_name)]
                    # split_conn = "attack_support"
                    # filename = "ibmcs_all.csv"
                    print(split_perc, split_conn, filename, feat_name)
                    overview_line = {"split_perc": split_perc, "split_conn": split_conn, "filename": filename, "feat_name": feat_name}
                    df_train = df[df["set"]== "train"]
                    if "dev" in set(df["set"]):
                        df_dev = df[df["set"]== "dev"]
                    elif "val" in set(df["set"]):
                        df_dev = df[df["set"]== "val"]
                    df_test = df[df["set"]== "test"]

                    # for i in range(5):
                    i = 0
                    if search_params:
                        study = optuna.create_study(direction="maximize", sampler=TPESampler())
                        study.optimize(objective_optuna, n_trials=250, show_progress_bar=False, timeout=600,  gc_after_trial=True, n_jobs=-1)
                        # study.optimize(detailed_objective_dev, n_trials=2, show_progress_bar=True)
                        # print(study.best_trial)
                        out_df = study.trials_dataframe()
                        out_df.loc[out_df['number'] == study.best_trial.number, 'best_trial'] = 1
                        out_df.loc[out_df['number'] != study.best_trial.number, 'best_trial'] = 0
                        overview_line.update(study.best_trial.params)

                        acc_dev, f1_dev, recall_dev, precision_dev, f1_per_label_dev, recall_per_label_dev, precision_per_label_dev = detailed_objective_dev(study.best_trial)
                        acc_test, f1_test, recall_test, precision_test, f1_per_label_test, recall_per_label_test, precision_per_label_test = detailed_objective_test(study.best_trial)
                    else:
                        acc_dev, f1_dev, recall_dev, precision_dev, f1_per_label_dev, recall_per_label_dev, precision_per_label_dev = train_eval_lightgbm(best_params, "dev")
                        acc_test, f1_test, recall_test, precision_test, f1_per_label_test, recall_per_label_test, precision_per_label_test = train_eval_lightgbm(best_params, "test")
                    print(acc_dev, f1_dev, recall_dev, precision_dev)
                    if search_params:
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_acc'] = acc_dev
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_recall_macro'] = recall_dev
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_precision_macro'] = precision_dev
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_f1_macro'] = f1_dev
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_recall_0'] = recall_per_label_dev[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_precision_0'] = precision_per_label_dev[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_f1_0'] = f1_per_label_dev[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_recall_1'] = recall_per_label_dev[1]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_precision_1'] = precision_per_label_dev[1]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'dev_f1_1'] = f1_per_label_dev[1]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'run'] = i
                    overview_line.update({"run": i, "f1_dev": f1_dev, "recall_dev": recall_dev, "precision_dev": precision_dev, "f1_dev_0": f1_per_label_dev[0], "recall_dev_0": recall_per_label_dev[0], "prec_dev_0": precision_per_label_dev[0], "f1_dev_1": f1_per_label_dev[1], "recall_dev_1": recall_per_label_dev[1], "prec_dev_1": precision_per_label_dev[1]})


                    print(acc_test, f1_test, recall_test, precision_test)
                    if search_params:
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_acc'] = acc_test
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_recall_macro'] = recall_test
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_precision_macro'] = precision_test
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_f1_macro'] = f1_test
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_recall_0'] = recall_per_label_test[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_precision_0'] = precision_per_label_test[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_f1_0'] = f1_per_label_test[0]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_recall_1'] = recall_per_label_test[1]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_precision_1'] = precision_per_label_test[1]
                        out_df.loc[out_df['number'] == study.best_trial.number, 'test_f1_1'] = f1_per_label_test[1]
                        out_df.to_csv("results/run"+str(run)+"_"+model_conns+"_"+filename+"_"+split_perc+"_"+split_conn+"_"+feat_name+".csv")
                    overview_line.update({"run": i, "f1_test": f1_test, "recall_test": recall_test, "precision_test": precision_test, "f1_test_0": f1_per_label_test[0], "recall_test_0": recall_per_label_test[0], "prec_test_0": precision_per_label_test[0], "f1_test_1": f1_per_label_test[1], "recall_test_1": recall_per_label_test[1], "prec_test_1": precision_per_label_test[1]})
                    best_results.append(overview_line)
                    pd.DataFrame(best_results).to_csv("results/overview.csv")

In [ ]:
out_df[out_df["best_trial"]==1]

In [ ]:
study.trials_dataframe()

In [ ]:
study.best_trial

In [ ]:
study.best_trial.params["classifier"]